In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import sys
import os
import torch as pt
from tqdm import tqdm

# Add the project root
sys.path.append(os.path.abspath('../../'))

from src.useful_functions import read_dirs_paths
from src.isokann.modules3 import *

# For matplotlib
font = {'size'   : 10}
plt.rc('font', **font)
in2cm = 1/2.54  # centimeters in inches

# For reproducibility
np.random.seed(0)
pt.manual_seed(0)

# Read directory paths
read_dirs_paths('dir_paths.txt', globals())

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")
print("")
print(device)


Created variables:
inp_dir = input/
out_trajectories1 = output/trajectories/
out_trajectories2 = output/trajectories/openmm_files/
out_trajectories3 = output/trajectories/openmm_files/initial_states/
out_trajectories4 = output/trajectories/openmm_files/final_states/
out_trajectories5 = output/trajectories/openmm_files/final_states2/
out_isokann = output/isokann/
out_mokito = output/mokito/

cuda


In [ ]:
# Load initial and final states and convert to torch
D0 = pt.load(out_trajectories1 + 'PWDistances_0.pt', map_location=device)
DT = pt.load(out_trajectories1 + 'PWDistances_t.pt', map_location=device)

Npoints = D0.shape[0]
Ndims   = D0.shape[1]

Nfinpoints  = DT.shape[1]
Nframes     = DT.shape[3]


frame = 0
Dt = pt.clone(DT[frame,:,:,:])

print(Dt.shape)

torch.Size([4000, 10, 66])


In [3]:
NN_epochs = [5, 10 ,15, 20]


NN_nodes =          [(Ndims, int(2*Ndims/3), 1), 
                      (Ndims, int(2*Ndims/3), int(Ndims/3), 1),
                      (Ndims, int(Ndims/2), 1), 
                      (Ndims, int(Ndims/2),int(Ndims/4), 1)]

NN_lr = [ 0.01,
          0.001,
          0.0001]

NN_wd  = [ 0.01,
           0.001,
           0.0001,
           0.00001]

NN_bs  = [50, 100, 200]

NN_patience = [2, 3, 5]

NN_act_fun = ['leakyrelu']

best_hyperparams, best_val_loss = random_search(D0,
                                                Dt,
                                                NN_epochs,
                                                NN_nodes,
                                                NN_lr,
                                                NN_wd,
                                                NN_bs,
                                                NN_patience,
                                                NN_act_fun,
                                                search_iterations=15,
                                                test_size = 0.2)

print("The best hyperparameters are:", best_hyperparams)
print("The best validation loss is:",  best_val_loss)


  0%|          | 0/15 [00:00<?, ?it/s]

 
Nepochs = 20
Nodes = [66 33 16  1]
Learning rate = 0.01
Weight decay = 0.0001
Batch size = 200
Patience = 3
Activation function = leakyrelu
Validation loss: 

  7%|▋         | 1/15 [00:11<02:39, 11.38s/it]

tensor(5.8871e-06, device='cuda:0')
Convergence: 1.0004278084162463
 
Nepochs = 15
Nodes = [66 33 16  1]
Learning rate = 0.001
Weight decay = 0.001
Batch size = 200
Patience = 2
Activation function = leakyrelu


 13%|█▎        | 2/15 [00:52<06:18, 29.14s/it]

...

Validation loss: tensor(2.9104e-05, device='cuda:0')
Convergence: 1.0005565291294412
 
Nepochs = 20
Nodes = [66 33  1]
Learning rate = 0.0001
Weight decay = 0.001
Batch size = 50
Patience = 5
Activation function = leakyrelu


 93%|█████████▎| 14/15 [11:06<01:01, 61.33s/it]

Validation loss: tensor(0.0392, device='cuda:0')
Convergence: 1.0139033656610594
 
Nepochs = 15
Nodes = [66 33 16  1]
Learning rate = 0.001
Weight decay = 0.0001
Batch size = 50
Patience = 3
Activation function = leakyrelu


100%|██████████| 15/15 [12:15<00:00, 49.01s/it]

Validation loss: tensor(1.5908e-05, device='cuda:0')
Convergence: 1.000868111035734
The best hyperparameters are: {'Nepochs': 20, 'nodes': array([66, 33, 16,  1]), 'learning_rate': 0.01, 'weight_decay': 0.0001, 'batch_size': 200, 'patience': 3, 'act_fun': 'leakyrelu'}
The best validation loss is: tensor(5.8871e-06, device='cuda:0')


In [4]:
import pickle
# Save to a file
with open(out_isokann + 'hyperparameters.pkl', 'wb') as file:
    pickle.dump(best_hyperparams, file)